<a href="https://colab.research.google.com/github/zabihin/Bloc1_Project_Kayak/blob/main/Bloc_1_Plan_your_trip_with_Kayak_Zahra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [ ]:
!pip install Scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Import libraries

In [ ]:
import datetime
import calendar
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
from statistics import mode
import plotly.express as px
import plotly.graph_objects as go



فرض کنیم یک زوج تصمیم گرفتند این آخر هفته رو به سفری ر فرانسه بروند. آنها می خواهند از بن 35 شهر در فرانسه یکی را بر اساس آب وهوا انتخاب کنند. من فرض کرده ام اگر میانگین دمای 35 شهر بالاتر از 22 باشد(احتمالا تابستان است) و ترجیح با خنک ترین مقصد است ولی اگر دما کم باشد ترجیح با گرم ترین شهر است. از بین 35 شهر ، 5 شهر با توجه به آب و هوا انتخاب می شود. 

find the date of next saturday and sunday for finding the weather and find hotels






In [ ]:
today = datetime.date.today() 
sat = today + datetime.timedelta((calendar.SATURDAY-today.weekday()) % 7 )
sun= today + datetime.timedelta((calendar.SUNDAY-today.weekday()) % 7 )
sun

datetime.date(2022, 10, 30)


this is the list of best cities to travel in france. 

In [ ]:
#city list
best_cities=["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg",
"Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas", "Cassis",
"Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
"Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

we should find the location of each city (we need them for finding how weather will be this weekend)

In [ ]:
url = "https://nominatim.openstreetmap.org/search?"

def find_gps(address, format='json'):
   
    params = {"q":address, "format": format}
    response = requests.get(url, params=params)
    return response.json()
columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(best_cities):
    response = find_gps(address = city)
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [ ]:
df_cities.head()


,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


find the temprature of this saturday and sunday 

my key for the website https://api.openweathermap.org

In [ ]:
key='2941f971df0a6e6e0569606b236025bf'

In [ ]:
df_cities['Temperature']=None
df_cities['Temperature_max']=None
df_cities['Temperature_min']=None


df_cities['Humidity']=None


for ind in df_cities.id:

    lat = df_cities.latitude[ind]
    lon = df_cities.longitude[ind]
    r =  requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&units=metric&appid={key}").json()
    weather = r['daily'][(sat.weekday()-today.weekday()):(sat.weekday()-today.weekday())+2]
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature')] =np.mean([i['feels_like']['day'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature_max')] =np.mean([i['temp']['max'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Temperature_min')] =np.mean([i['temp']['min'] for i in weather])
    df_cities.iat[ind, df_cities.columns.get_loc('Humidity')] =np.mean( [i['humidity'] for i in weather])



In [ ]:
weather

[{'dt': 1667041200,
  'sunrise': 1667025645,
  'sunset': 1667062543,
  'moonrise': 1667043600,
  'moonset': 1667072220,
  'moon_phase': 0.14,
  'temp': {'day': 19.95,
   'min': 17.97,
   'max': 21.12,
   'night': 18.78,
   'eve': 18.9,
   'morn': 17.97},
  'feels_like': {'day': 19.8, 'night': 18.44, 'eve': 18.73, 'morn': 18.02},
  'pressure': 1019,
  'humidity': 69,
  'dew_point': 13.82,
  'wind_speed': 7.22,
  'wind_deg': 193,
  'wind_gust': 12.73,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': 100,
  'pop': 0,
  'uvi': 0},
 {'dt': 1667127600,
  'sunrise': 1667112132,
  'sunset': 1667148851,
  'moonrise': 1667133840,
  'moonset': 1667162640,
  'moon_phase': 0.18,
  'temp': {'day': 23.04,
   'min': 16.24,
   'max': 24.13,
   'night': 20.72,
   'eve': 21.66,
   'morn': 16.24},
  'feels_like': {'day': 22.73, 'night': 20.31, 'eve': 21.32, 'morn': 15.93},
  'pressure': 1012,
  'humidity': 51,
  'dew_point': 11.99,
  'w

In [ ]:
df_cities

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,18.84,20.105,13.27,62.5
1,1,St Malo,48.649518,-2.0260409,18.225,19.575,13.86,66.5
2,2,Bayeux,49.2764624,-0.7024738,19.11,19.57,13.335,58.5
3,3,Le Havre,49.4938975,0.1079732,18.065,18.89,15.015,69.0
4,4,Rouen,49.4404591,1.0939658,19.76,20.485,13.69,57.0
5,5,Paris,48.8588897,2.3200410217200766,21.835,22.96,17.235,46.0
6,6,Amiens,49.8941708,2.2956951,19.685,20.45,14.245,61.0
7,7,Lille,50.6365654,3.0635282,19.01,19.92,14.155,67.5
8,8,Strasbourg,48.584614,7.7507127,19.16,19.625,12.97,58.0
9,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,17.835,18.565,11.29,52.0


find the avrerage of tempreture(feel-like,max temp, min temp) in this week-end for list of cities
if the mean of tempreture is more than 22, I assume that the best destination are the ones which have minimum of max temrature in that day 
and 
if the mean of tempreture is less than 22, I assume that the best destination are the ones which have maximum of min temrature in that day 

In [ ]:
Tem_avr=df_cities['Temperature'].mean()

In [ ]:
df_cities['rank_weather']=None

if (Tem_avr-22)>0:
  df_cities.loc[:, 'rank_weather'] = df_cities['Temperature_max'].rank(method='min').astype(int)
  df_cities = df_cities.sort_values(by=['Temperature_max'], ascending=[True]).reset_index(drop=True)
else:
  df_cities.loc[:, 'rank_weather'] = df_cities['Temperature_min'].rank(method='max').astype(int)
  df_cities = df_cities.sort_values(by=['Temperature_min'], ascending=[True]).reset_index(drop=True)



In [ ]:
df_cities

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity,rank_weather
0,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,17.835,18.565,11.29,52.0,1
1,29,Ariege,42.9455368,1.4065544156065486,21.735,22.81,11.99,22.5,2
2,13,Dijon,47.3215806,5.0414701,20.54,21.085,12.255,49.5,3
3,17,Gorges du Verdon,43.7496562,6.3285616,20.645,21.43,12.31,39.0,4
4,8,Strasbourg,48.584614,7.7507127,19.16,19.625,12.97,58.0,5
5,23,Uzes,44.0121279,4.4196718,21.66,22.81,13.22,49.5,6
6,14,Annecy,45.8992348,6.1288847,21.36,21.825,13.225,49.5,7
7,0,Mont Saint Michel,48.6359541,-1.511459954959514,18.84,20.105,13.27,62.5,8
8,12,Besancon,47.2380222,6.0243622,22.18,22.56,13.28,50.0,9
9,2,Bayeux,49.2764624,-0.7024738,19.11,19.57,13.335,58.5,10


Top 5 cities based on the weather

In [ ]:
top5_city=df_cities[0:5]

In [ ]:
top5_city

,id,name,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity,rank_weather
0,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195,17.835,18.565,11.29,52.0,1
1,29,Ariege,42.9455368,1.4065544156065486,21.735,22.81,11.99,22.5,2
2,13,Dijon,47.3215806,5.0414701,20.54,21.085,12.255,49.5,3
3,17,Gorges du Verdon,43.7496562,6.3285616,20.645,21.43,12.31,39.0,4
4,8,Strasbourg,48.584614,7.7507127,19.16,19.625,12.97,58.0,5


In [ ]:
best_5city=top5_city.name.to_list()

In [ ]:
best_5city

['Chateau du Haut Koenigsbourg',
 'Ariege',
 'Dijon',
 'Gorges du Verdon',
 'Strasbourg']

the date of travel for scraping booking.com

*   2 person with no kid
*   date next weekend just for one night
*   order of hotel : based of review_score_and_price






In [ ]:
year_in=str(sat.year)
month_in=str(sat.month)
day_in=str(sat.day)
month_out=str(sun.month)
day_out=str(sun.day)

In [ ]:
class BookSpider(scrapy.Spider):
    name = "hotels"
    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']
    
    
    # Parse function for login
    def parse(self, response):
      for city in best_5city:
        # FormRequest used to login
        yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city,"checkin_year":year_in,"checkin_month":month_in,"checkin_monthday":day_in,
            "checkout_year":year_in,"checkout_month":month_out,"checkout_monthday":day_out,"selected_currency":"EUR",
            "group_adults":"2","order":"review_score_and_price",'nflt':'%3Bht_id%3D204'},
            callback=self.after_search
        )

    # Callback used after login
    def after_search(self, response):
      for item in response.css("div.main-spacing"):
        yield {
                  'city': item.css("div.efdb2b543b h1.e1f827110f.d3a14d00da::text").getall()[0].split("\xa0")[0],
                  'hotel': item.css("div.fcab3ed991.a23c043802::text").getall(),
                  'url':item.css("h3.a4225678b2 a.e13098a59f::attr(href) ").getall(),
                  'price': item.css("span.bd73d13072.fcab3ed991::text").getall(),
                  'rating':item.css("div.b5cd09854e.d10a6220b4::text").getall(),
                  }




In [ ]:
filename = "book.json"

if filename in os.listdir():
        os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename: {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(BookSpider)
process.start()


INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-24 16:52:08 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-24 16:52:08 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) '
               'Gecko/20100101 Firefox/92.0'}
2022-10-24 16:5

In [ ]:
df = pd.read_json('book.json')


list of hotels, their webadress, price and rating for 5 cites

In [ ]:
df

,city,hotel,url,price,rating
0,Château du Haut-Kœnigsbourg,"[Hôtel du Haut Koenigsbourg - calme et nature,...",[https://www.booking.com/hotel/fr/le-haut-koen...,"[€ 125, € 429, € 248, € 138, € 1 128, € 70, € ...","[9,0, 8,5, 8,0, 7,9, 9,2, 8,0, 8,2, 7,7, 7,9, ..."
1,Ariège,"[Le Clos Cathala, Relais Mira Peis, Hôtel Conf...",[https://www.booking.com/hotel/fr/le-clos-cath...,"[€ 220, € 251, € 60, € 74, € 152, € 230, € 60,...","[9,1, 9,1, 8,5, 8,8, 8,6, 8,8, 8,2, 8,0, 8,2, ..."
2,Dijon,[ibis Styles Dijon Sud Domaine De Beauregard 2...,[https://www.booking.com/hotel/fr/ibis-styles-...,"[€ 129, € 161, € 202, € 85, € 89, € 100, € 103...","[8,7, 8,6, 8,7, 8,1, 8,0, 8,1, 8,3, 8,0, 8,3, ..."
3,Gorges du Verdon,"[Le Moulin du Verdon, Hotel Les Restanques De ...",[https://www.booking.com/hotel/fr/residence-le...,"[€ 98, € 122, € 351, € 75, € 97, € 103, € 104,...","[9,1, 9,0, 9,1, 8,5, 8,7, 8,5, 8,5, 8,7, 8,7, ..."
4,Strasbourg,"[Boutique Hotel Des XV, Aloft Strasbourg Etoil...",[https://www.booking.com/hotel/fr/des-xv.fr.ht...,"[€ 235, € 269, € 305, € 213, € 324, € 141, € 1...","[9,0, 8,7, 8,8, 8,3, 8,1, 8,7, 8,6, 8,1, 7,9, ..."


adding the other infomation of gps and weather to this dataframe


In [ ]:
df["latitude"]=top5_city["latitude"]
df["longitude"]=top5_city["longitude"]
df["Temperature"]=top5_city["Temperature"]
df["Temperature_max"]=top5_city["Temperature_max"]
df["Temperature_min"]=top5_city["Temperature_min"]
df["Humidity"]=top5_city["Humidity"]

df

,city,hotel,url,price,rating,latitude,longitude,Temperature,Temperature_max,Temperature_min,Humidity
0,Château du Haut-Kœnigsbourg,"[Hôtel du Haut Koenigsbourg - calme et nature,...",[https://www.booking.com/hotel/fr/le-haut-koen...,"[€ 125, € 429, € 248, € 138, € 1 128, € 70, € ...","[9,0, 8,5, 8,0, 7,9, 9,2, 8,0, 8,2, 7,7, 7,9, ...",48.249489800000006,7.34429620253195,17.835,18.565,11.29,52.0
1,Ariège,"[Le Clos Cathala, Relais Mira Peis, Hôtel Conf...",[https://www.booking.com/hotel/fr/le-clos-cath...,"[€ 220, € 251, € 60, € 74, € 152, € 230, € 60,...","[9,1, 9,1, 8,5, 8,8, 8,6, 8,8, 8,2, 8,0, 8,2, ...",42.9455368,1.4065544156065486,21.735,22.81,11.99,22.5
2,Dijon,[ibis Styles Dijon Sud Domaine De Beauregard 2...,[https://www.booking.com/hotel/fr/ibis-styles-...,"[€ 129, € 161, € 202, € 85, € 89, € 100, € 103...","[8,7, 8,6, 8,7, 8,1, 8,0, 8,1, 8,3, 8,0, 8,3, ...",47.3215806,5.0414701,20.54,21.085,12.255,49.5
3,Gorges du Verdon,"[Le Moulin du Verdon, Hotel Les Restanques De ...",[https://www.booking.com/hotel/fr/residence-le...,"[€ 98, € 122, € 351, € 75, € 97, € 103, € 104,...","[9,1, 9,0, 9,1, 8,5, 8,7, 8,5, 8,5, 8,7, 8,7, ...",43.7496562,6.3285616,20.645,21.43,12.31,39.0
4,Strasbourg,"[Boutique Hotel Des XV, Aloft Strasbourg Etoil...",[https://www.booking.com/hotel/fr/des-xv.fr.ht...,"[€ 235, € 269, € 305, € 213, € 324, € 141, € 1...","[9,0, 8,7, 8,8, 8,3, 8,1, 8,7, 8,6, 8,1, 7,9, ...",48.584614,7.7507127,19.16,19.625,12.97,58.0


In [ ]:
best1={"name":df.city[0],"hotel":df.hotel[0][0:20],"url":df.url[0][0:20],"price":df.price[0][0:20],"rating":df.rating[0][0:20],
       "Temperature":df.Temperature[0],"Temperature_max":df.Temperature_max[0],"Temperature_min":df.Temperature_min[0],"Humidity":df.Humidity[0]
       }
df_1=pd.DataFrame(best1)
df_1


,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity
0,Château du Haut-Kœnigsbourg,Hôtel du Haut Koenigsbourg - calme et nature,https://www.booking.com/hotel/fr/le-haut-koeni...,€ 125,"9,0",17.835,18.565,11.29,52.0
1,Château du Haut-Kœnigsbourg,Hôtel Barrière Ribeauvillé,https://www.booking.com/hotel/fr/resort-barrie...,€ 429,"8,5",17.835,18.565,11.29,52.0
2,Château du Haut-Kœnigsbourg,Caveau de l'ami Fritz,https://www.booking.com/hotel/fr/caveau-de-l-a...,€ 248,"8,0",17.835,18.565,11.29,52.0
3,Château du Haut-Kœnigsbourg,"Hôtel Restaurant Le Verger des Châteaux, The O...",https://www.booking.com/hotel/fr/le-verger-des...,€ 138,"7,9",17.835,18.565,11.29,52.0
4,Château du Haut-Kœnigsbourg,Hostellerie La Cheneaudière & Spa,https://www.booking.com/hotel/fr/hostellerie-l...,€ 1 128,"9,2",17.835,18.565,11.29,52.0
5,Château du Haut-Kœnigsbourg,Hotel Marchal,https://www.booking.com/hotel/fr/marchal.fr.ht...,€ 70,"8,0",17.835,18.565,11.29,52.0
6,Château du Haut-Kœnigsbourg,Logis Hôtel Restaurant La Tête Des Faux,https://www.booking.com/hotel/fr/restaurant-la...,€ 95,"8,2",17.835,18.565,11.29,52.0
7,Château du Haut-Kœnigsbourg,Hotel Restaurant Du Kalblin,https://www.booking.com/hotel/fr/restaurant-du...,€ 63,"7,7",17.835,18.565,11.29,52.0
8,Château du Haut-Kœnigsbourg,Hôtel Le Manoir,https://www.booking.com/hotel/fr/manoir.fr.htm...,€ 178,"7,9",17.835,18.565,11.29,52.0
9,Château du Haut-Kœnigsbourg,Centre International de Sejour - Le Mittel,https://www.booking.com/hotel/fr/ethic-etape-c...,€ 60,"7,1",17.835,18.565,11.29,52.0


In [ ]:
best2={"name":df.city[1],"hotel":df.hotel[1][0:20],"url":df.url[1][0:20],"price":df.price[1][0:20],"rating":df.rating[1][0:20],
       "Temperature":df.Temperature[1],"Temperature_max":df.Temperature_max[1],"Temperature_min":df.Temperature_min[1],"Humidity":df.Humidity[1]}
df_2=pd.DataFrame(best2)

In [ ]:
best3={"name":df.city[2],"hotel":df.hotel[2][0:20],"url":df.url[2][0:20],"price":df.price[2][0:20],"rating":df.rating[2][0:20],
       "Temperature":df.Temperature[2],"Temperature_max":df.Temperature_max[2],"Temperature_min":df.Temperature_min[2],"Humidity":df.Humidity[2]}
df_3=pd.DataFrame(best3)

In [ ]:
best4={"name":df.city[3],"hotel":df.hotel[3][0:20],"url":df.url[3][0:20],"price":df.price[3][0:20],"rating":df.rating[3][0:20],
       "Temperature":df.Temperature[3],"Temperature_max":df.Temperature_max[3],"Temperature_min":df.Temperature_min[3],"Humidity":df.Humidity[3]}
df_4=pd.DataFrame(best4)

In [ ]:
best5={"name":df.city[4],"hotel":df.hotel[4][0:20],"url":df.url[4][0:20],"price":df.price[4][0:20],"rating":df.rating[4][0:20],
       "Temperature":df.Temperature[4],"Temperature_max":df.Temperature_max[4],"Temperature_min":df.Temperature_min[4],"Humidity":df.Humidity[4]}
df_5=pd.DataFrame(best5)

In [ ]:
df_topp5=df_1.append((df_2,df_3,df_4,df_5), ignore_index = True)


In [ ]:
df_topp5

,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity
0,Château du Haut-Kœnigsbourg,Hôtel du Haut Koenigsbourg - calme et nature,https://www.booking.com/hotel/fr/le-haut-koeni...,€ 125,"9,0",17.835,18.565,11.29,52.0
1,Château du Haut-Kœnigsbourg,Hôtel Barrière Ribeauvillé,https://www.booking.com/hotel/fr/resort-barrie...,€ 429,"8,5",17.835,18.565,11.29,52.0
2,Château du Haut-Kœnigsbourg,Caveau de l'ami Fritz,https://www.booking.com/hotel/fr/caveau-de-l-a...,€ 248,"8,0",17.835,18.565,11.29,52.0
3,Château du Haut-Kœnigsbourg,"Hôtel Restaurant Le Verger des Châteaux, The O...",https://www.booking.com/hotel/fr/le-verger-des...,€ 138,"7,9",17.835,18.565,11.29,52.0
4,Château du Haut-Kœnigsbourg,Hostellerie La Cheneaudière & Spa,https://www.booking.com/hotel/fr/hostellerie-l...,€ 1 128,"9,2",17.835,18.565,11.29,52.0
...,...,...,...,...,...,...,...,...,...
95,Strasbourg,ibis Strasbourg Sud La Vigie,https://www.booking.com/hotel/fr/ha-tel-ibis-s...,€ 84,"7,3",19.160,19.625,12.97,58.0
96,Strasbourg,B&B HOTEL Strasbourg Nord Schiltigheim,https://www.booking.com/hotel/fr/ibis-budget-s...,€ 89,"6,2",19.160,19.625,12.97,58.0
97,Strasbourg,ATOLON PARK HOTEL,https://www.booking.com/hotel/fr/atolon-park.f...,€ 79,"8,7",19.160,19.625,12.97,58.0
98,Strasbourg,Hôtel Restaurant Oberlé,https://www.booking.com/hotel/fr/ha-tel-restau...,€ 90,"8,8",19.160,19.625,12.97,58.0


In [ ]:
df_topp5['lon_hotel']=None
df_topp5['lat_hotel']=None
df_topp5['address_hotel']=None

In [ ]:
for ind in df_topp5.index:
  print(ind)
  url=df_topp5.url[ind]
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "lxml")
  df_topp5.lat_hotel[ind]=soup.find( class_ = "jq_tooltip loc_block_link_underline_fix bui-link show_on_map_hp_link show_map_hp_link " ).get("data-atlas-latlng").split(',')[0]
  df_topp5.lon_hotel[ind]=soup.find( class_ = "jq_tooltip loc_block_link_underline_fix bui-link show_on_map_hp_link show_map_hp_link " ).get("data-atlas-latlng").split(',')[1]
  df_topp5.address_hotel[ind]=soup.find(class_ = " hp_address_subtitle js-hp_address_subtitle jq_tooltip ").getText().strip('\n')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


0


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/le-haut-koenigsbourg.fr.html?aid=304142&label=gen173bo-1DCAQoggJCI3NlYXJjaF9jaGF0ZWF1IGR1IGhhdXQga29lbmlnc2JvdXJnSA1YA2jnAYgBAZgBDbgBGMgBD9gBA-gBAfgBA4gCAZgCAqgCBLgCwIXbmgbAAgHSAiRkNTExNWQ0My1kNTQ4LTRiYzctYjY5Yy1mMTU2MzY5ODlmNzfYAgTgAgE&ucfs=1&arphpl=1&checkin=2022-10-29&checkout=2022-10-30&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=review_score_and_price&nflt=ht_id%3D204&srpvid=ce0676a01b9e03b8&srepoch=1666630337&all_sr_blocks=32124602_347512012_2_0_0&highlighted_blocks=32124602_347512012_2_0_0&matching_block_id=32124602_347512012_2_0_0&sr_pri_blocks=32124602_347512012_2_0_0__12500&from_sustainable_property_sr=1&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/le-haut-koenigsbourg.fr.html HTTP/1.1" 200 279370
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A valu

1


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/resort-barriere-ribeauville.fr.html?aid=304142&label=gen173bo-1DCAQoggJCI3NlYXJjaF9jaGF0ZWF1IGR1IGhhdXQga29lbmlnc2JvdXJnSA1YA2jnAYgBAZgBDbgBGMgBD9gBA-gBAfgBA4gCAZgCAqgCBLgCwIXbmgbAAgHSAiRkNTExNWQ0My1kNTQ4LTRiYzctYjY5Yy1mMTU2MzY5ODlmNzfYAgTgAgE&ucfs=1&arphpl=1&checkin=2022-10-29&checkout=2022-10-30&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=review_score_and_price&nflt=ht_id%3D204&srpvid=ce0676a01b9e03b8&srepoch=1666630337&all_sr_blocks=36438009_88549540_0_2_0&highlighted_blocks=36438009_88549540_0_2_0&matching_block_id=36438009_88549540_0_2_0&sr_pri_blocks=36438009_88549540_0_2_0__42907&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/resort-barriere-ribeauville.fr.html HTTP/1.1" 200 285339
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


...

DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/atolon-park.fr.html?aid=304142&label=gen173bo-1DCAQoggJCEXNlYXJjaF9zdHJhc2JvdXJnSA1YA2jnAYgBAZgBDbgBGMgBD9gBA-gBAfgBA4gCAZgCAqgCBLgCyIXbmgbAAgHSAiQ4N2VjMzYyMi0zZWU0LTQzZjEtYTNiMy1iYzg0MTFlNTlkNDnYAgTgAgE&ucfs=1&arphpl=1&checkin=2022-10-29&checkout=2022-10-30&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=18&hapos=18&sr_order=review_score_and_price&nflt=ht_id%3D204&srpvid=5dde76a42a5104de&srepoch=1666630345&all_sr_blocks=504275401_172879755_2_2_0&highlighted_blocks=504275401_172879755_2_2_0&matching_block_id=504275401_172879755_2_2_0&sr_pri_blocks=504275401_172879755_2_2_0__7900&from_sustainable_property_sr=1&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/atolon-park.fr.html HTTP/1.1" 200 278892
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


98


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/ha-tel-restaurant-oberla-c.fr.html?aid=304142&label=gen173bo-1DCAQoggJCEXNlYXJjaF9zdHJhc2JvdXJnSA1YA2jnAYgBAZgBDbgBGMgBD9gBA-gBAfgBA4gCAZgCAqgCBLgCyIXbmgbAAgHSAiQ4N2VjMzYyMi0zZWU0LTQzZjEtYTNiMy1iYzg0MTFlNTlkNDnYAgTgAgE&ucfs=1&arphpl=1&checkin=2022-10-29&checkout=2022-10-30&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=19&hapos=19&sr_order=review_score_and_price&nflt=ht_id%3D204&srpvid=5dde76a42a5104de&srepoch=1666630345&all_sr_blocks=37454201_226095474_2_2_0&highlighted_blocks=37454201_226095474_2_2_0&matching_block_id=37454201_226095474_2_2_0&sr_pri_blocks=37454201_226095474_2_2_0__9000&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/ha-tel-restaurant-oberla-c.fr.html HTTP/1.1" 200 271236
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.booking.com:443


99


DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/best-western-ha-tel-le-rha-c-nan.fr.html?aid=304142&label=gen173bo-1DCAQoggJCEXNlYXJjaF9zdHJhc2JvdXJnSA1YA2jnAYgBAZgBDbgBGMgBD9gBA-gBAfgBA4gCAZgCAqgCBLgCyIXbmgbAAgHSAiQ4N2VjMzYyMi0zZWU0LTQzZjEtYTNiMy1iYzg0MTFlNTlkNDnYAgTgAgE&ucfs=1&arphpl=1&checkin=2022-10-29&checkout=2022-10-30&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=20&hapos=20&sr_order=review_score_and_price&nflt=ht_id%3D204&srpvid=5dde76a42a5104de&srepoch=1666630345&all_sr_blocks=35847609_244920709_2_2_0&highlighted_blocks=35847609_244920709_2_2_0&matching_block_id=35847609_244920709_2_2_0&sr_pri_blocks=35847609_244920709_2_2_0__10900&from=searchresults HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.booking.com:443 "GET /hotel/fr/best-western-ha-tel-le-rha-c-nan.fr.html HTTP/1.1" 200 284163


In [ ]:
df_topp5.to_csv('/content/out.csv')  

In [ ]:
df_topp5=pd.read_csv('/content/out.csv')

In [ ]:
df_topp5.head()

,Unnamed: 0,name,hotel,url,price,rating,Temperature,Temperature_max,Temperature_min,Humidity,lon_hotel,lat_hotel,address_hotel
0,0,Château du Haut-Kœnigsbourg,Hôtel du Haut Koenigsbourg - calme et nature,https://www.booking.com/hotel/fr/le-haut-koeni...,€ 125,"9,0",17.835,18.565,11.29,52.0,7.302220,48.231641,"9 rue de Rodern, 68590 Thannenkirch, France"
1,1,Château du Haut-Kœnigsbourg,Hôtel Barrière Ribeauvillé,https://www.booking.com/hotel/fr/resort-barrie...,€ 429,"8,5",17.835,18.565,11.29,52.0,7.349178,48.187544,"RD 106 BP 50002, 68151 Ribeauvillé, France"
2,2,Château du Haut-Kœnigsbourg,Caveau de l'ami Fritz,https://www.booking.com/hotel/fr/caveau-de-l-a...,€ 248,"8,0",17.835,18.565,11.29,52.0,7.320242,48.194451,"1 Place de l'Ancien Hôpital, 68150 Ribeauvillé..."
3,3,Château du Haut-Kœnigsbourg,"Hôtel Restaurant Le Verger des Châteaux, The O...",https://www.booking.com/hotel/fr/le-verger-des...,€ 138,"7,9",17.835,18.565,11.29,52.0,7.419473,48.306555,"2 Route Romaine, 67650 Dieffenthal, France"
4,4,Château du Haut-Kœnigsbourg,Hostellerie La Cheneaudière & Spa,https://www.booking.com/hotel/fr/hostellerie-l...,€ 1 128,"9,2",17.835,18.565,11.29,52.0,7.183709,48.391124,"3 Rue Du Vieux Moulin, 67420 Colroy-la-Roche, ..."


In [ ]:
df_topp5.rating=df_topp5.rating.apply(lambda x: x.replace(",","."))
df_topp5.price=df_topp5.price.apply(lambda x: x.split("€")[1])



In [ ]:
df_topp5.price=df_topp5.price.apply(lambda x: x.replace("\xa01",''))


In [ ]:
df_topp5 = df_topp5.astype({"lat_hotel": float,"lon_hotel": float,"Temperature":float,"price":float})

In [ ]:
fig = px.scatter_mapbox(df_topp5, lat="lat_hotel", lon="lon_hotel",
                  
                            mapbox_style="carto-positron",zoom=4, width = 1000,  height =800,title = 'Location of top hotels in 20 different cities',color='Temperature',size='price'
                          
            )
fig.show(renderer="colab")